In [1]:
import pandas as pd
import json
import matplotlib.pyplot as plt

In [2]:
%matplotlib notebook

In [3]:
def collapse(x): 
    return x.replace(" ", "") 

In [4]:
state_abbrv = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}
# from http://code.activestate.com/recipes/577305-python-dictionary-of-us-states-and-territories/

In [5]:
#bring in medicare hospital data
data = pd.read_csv('Hospital_General_Information.csv')
hospital_df = pd.DataFrame(data)

In [6]:
#bring in data about the number of hospital beds
with open('beds.json') as jsonfile:
    beds_json = json.load(jsonfile)

In [7]:
#convert bed data to pared data frame
bedI = pd.DataFrame(beds_json)
bed_df=bedI[['hospital_bed_count','medicare_provider_number','fips_state_and_county_code']]
bed_df = bed_df.rename(columns={'medicare_provider_number':'Facility ID'})

In [8]:
#remove extreneous rows from hospital data
del hospital_df['Hospital overall rating footnote']
del hospital_df['Mortality national comparison footnote']
del hospital_df['Safety of care national comparison footnote']
del hospital_df['Readmission national comparison footnote']
del hospital_df['Patient experience national comparison footnote']
del hospital_df['Efficient use of medical imaging national comparison footnote']
del hospital_df['Effectiveness of care national comparison footnote']
del hospital_df['Timeliness of care national comparison footnote']
del hospital_df['Address']
del hospital_df['City']
del hospital_df['Hospital Ownership']
del hospital_df['Phone Number']



In [9]:
df = pd.merge(hospital_df, bed_df, on = 'Facility ID', how = 'left')
#merge the bed data and the medicare data

In [10]:
#isolate only hospitals that have a rating
df2 = df.loc[df['Hospital overall rating']!='Not Available']
df2 =df2.astype({'Hospital overall rating':'int64'})
#create a list of states to cycle through
states = df2['State'].value_counts()

In [11]:
#bring in fips data to fill in for counties in covid data set
data = pd.read_csv("https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv")
death_data = pd.DataFrame(data)
fips_death = death_data[['state','county','fips']]
#add a test ccolumn to improve matches to fill in fips
countyser = []
for x,y in fips_death.iterrows():
    store = y['county']
    store = str(store).lower()
    store = collapse(store)
    countyser.append(store)
fips_death['test county']=countyser

C:\Users\littl\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [12]:
#sequentially determine avg values for hospital rating and emergency services
averages ={}
for st in states.index: #isolate each state first
    hold_df = df2.loc[df2['State']==st]  #assign state only df to a holding df
    counties = hold_df['County Name'].value_counts()# generate a list of counties to cycle
    averages.update( {st : {}} ) #initialize a subdictionary to store results in
    #series initialization for desired fields
    averages[st].update({'County':[],'Average Rating':[],'Emergency Services':[]}) 
    for cnt in counties.index:
        county = hold_df.loc[(df['County Name'] == cnt)]
        avg = county.mean()
        averages[st]['County'].append(cnt)  #store desired values in the seires
        averages[st]['Average Rating'].append(avg['Hospital overall rating'])
        averages[st]['Emergency Services'].append(avg['Emergency Services'])    

In [13]:
#sequentially determine cumulative values for hospital rating and emergency services
#see above for infromation about structure
sums = {}
df3 = df[['County Name','State','hospital_bed_count']]
df3 = df3.dropna()
df3 = df3.astype({'hospital_bed_count':'int64'})
for st in states.index:
    hold_df = df3.loc[df3['State']==st]
    counties = hold_df['County Name'].value_counts()
    sums.update( {st : {}} )
    sums[st].update({'County':[],'County Beds':[]})
    for cnt in counties.index:
        county = hold_df.loc[(df['County Name'] == cnt)]
        totals = county.sum()
        sums[st]['County'].append(cnt)
        sums[st]['County Beds'].append(totals['hospital_bed_count'])  

In [14]:
#sequentially determine counted values for hospital rating and emergency services
#see above for infromation about structure
counts = {}
for st in states.index:
    hold_df = df.loc[df['State']==st]
    counties = hold_df['County Name'].value_counts()
    counts.update( {st : {}} )
    counts[st].update({'County':[],'Hospitals':[],'fips':[]})
    for cnt in counties.index:
        county = hold_df.loc[(df['County Name'] == cnt)]
        totals = county.count()
        counts[st]['County'].append(cnt)
        counts[st]['Hospitals'].append(totals['Facility ID'])
        #for each county determine the fips id using beds dataframe
        fips_df = county.dropna(subset=['fips_state_and_county_code'])  #store all counties that have a fips id
        fips_df = fips_df.reset_index() 
        try:
            counts[st]['fips'].append(fips_df['fips_state_and_county_code'][0])
        except:# if no fips data in the original dataframe. THis uses a search from the covid data
            try:
                county = collapse(cnt).lower()
                state = state_abbrv[st]
                hold = fips_death.loc[(fips_death['test county']==county)&(fips_death['state']==state)]
                hold = hold.reset_index()
                counts[st]['fips'].append(hold['fips'][0])
            except:
                counts[st]['fips'].append(0)#add a zero place holder for any county without fips data
                print(f'{county} {state}')
         

trinity California
mariposa California
upton Texas
runnels Texas
hardeman Texas
wheeler Texas
coleman Texas
wilbarger Texas
collingsworth Texas
culberson Texas
throckmorton Texas
bosque Texas
ochiltree Texas
schleicher Texas
fisher Texas
cochran Texas
stonewall Texas
ward Texas
sutton Texas
brewster Texas
nolan Texas
houston Texas
baylor Texas
reagan Texas
refugio Texas
childress Texas
bailey Texas
dimmit Texas
reeves Texas
somervell Texas
haskell Texas
kimble Texas
union Illinois
mcdonough Illinois
hamilton Illinois
clay Illinois
northumberlnd Pennsylvania
hocking Ohio
baraga Michigan
iron Missouri
sullivan Missouri
barton Missouri
dent Missouri
putnam Missouri
fleming Kentucky
cass Iowa
monroe Iowa
lucas Iowa
wayne Iowa
humboldt Iowa
decatur Iowa
floyd Iowa
davis Iowa
paloalto Iowa
pocahontas Iowa
ringgold Iowa
st.johnbaptist Louisiana
ness Kansas
wilson Kansas
dickinson Kansas
washington Kansas
ellsworth Kansas
lane Kansas
kearny Kansas
stanton Kansas
wichita Kansas
lincoln Kansas
r

In [15]:
#initialize a blank dataframe
hold_df = pd.DataFrame({'County':[],'State':[],'Average Rating':[],'Hospitals':[],
                        'Emergency Services':[],'County Beds':[]})
for st in states.index:  #cycle through each state
    count_state = pd.DataFrame(counts[st])  #convert the dictionaries into individual dataframes
    avg_state = pd.DataFrame(averages[st])
    sum_state = pd.DataFrame(sums[st])
    state_df = pd.merge(count_state,avg_state,on = 'County',how= 'outer')  #merge dataframes
    state_df = pd.merge(state_df,sum_state,on = 'County',how='outer')
    state_df['State'] = st  #fill in a state column for each county
    final_df = pd.concat([hold_df,state_df])  #add the current state df to a running df
    hold_df = final_df  #hold df and final df needed to keep from recursion errors
final_df =final_df.astype({'fips':'int64'})
final_df.to_csv('output_data/UShospitals.csv')
final_df.head()



,County,State,Average Rating,Hospitals,Emergency Services,County Beds,fips
0,LOS ANGELES,CA,2.902778,90.0,0.583333,15072.0,6037
1,ORANGE,CA,3.100000,28.0,0.900000,3054.0,6059
2,SAN BERNARDINO,CA,2.400000,22.0,0.933333,3230.0,6071
3,SAN DIEGO,CA,4.000000,21.0,0.933333,4205.0,6073
4,RIVERSIDE,CA,2.437500,17.0,0.937500,987.0,6065


In [16]:
NC = final_df.loc[final_df['State']=='NC']
NC.to_csv('output_data/NCHospitalData.csv')
NC.head(30)

,County,State,Average Rating,Hospitals,Emergency Services,County Beds,fips
0,MECKLENBURG,NC,3.833333,8.0,1.0,1054.0,37119
1,WAKE,NC,4.000000,6.0,1.0,1477.0,37183
2,FORSYTH,NC,3.000000,4.0,1.0,1933.0,37067
3,DURHAM,NC,4.500000,3.0,1.0,1410.0,37063
4,ONSLOW,NC,2.000000,3.0,1.0,NaN,37133
5,GRANVILLE,NC,3.000000,3.0,1.0,NaN,37077
6,BRUNSWICK,NC,3.000000,3.0,1.0,60.0,37019
7,IREDELL,NC,2.666667,3.0,1.0,105.0,37097
8,BUNCOMBE,NC,5.000000,2.0,1.0,735.0,37021
9,NEW HANOVER,NC,3.000000,2.0,1.0,NaN,37129


In [17]:
# fips_hold=[]
# for x,y in final_df.iterrows():
#     county = collapse(y['County']).lower()
#     state = state_abbrv[y['State']]
#     hold = fips_df.loc[(fips_df['test county']==county)&(fips_df['state']==state)]
#     hold = hold.reset_index()
#     try:
#         fips_hold.append(hold['fips'][0])
#     except:
#         try:
#             county2 = county + 'county'
#             hold = secondfips.loc[(secondfips['test county']==county2)&(secondfips['state']==y['State'])]
#             hold = hold.reset_index()
#             fips_hold.append(hold['fips'][0])
#         except:
#             try:
#                 county2 = collapse(county) +'borough'
#                 hold = secondfips.loc[(secondfips['test county']==county2)&(secondfips['state']==y['State'])]
#                 hold = hold.reset_index()
#                 fips_hold.append(hold['fips'][0])
#             except:
#                 try:
#                     county2 = collapse(county)
#                     hold = secondfips.loc[(secondfips['test county']==county2)&(secondfips['state']==y['State'])]
#                     hold = hold.reset_index()
#                     fips_hold.append(hold['fips'][0])
#                      except:
#         fips_hold.append('0')
#             print(f'{county2},{state} did not work')
                

In [18]:
# dfar = final_df.loc[final_df['fips']<1000]
# # dfar = dfar.dropna(subset=['County Beds'])
# dfar.head(30)